# Debug 1
`Lambdad5_rinv2` and `Lambdad5_rinv10` have bug for analysis in `scheme_2_bug` folder.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
import myhep_old.analysis_v3 as myAnal_v3_old
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class
Use `Lambdad = 5` and `rinv = 0.2` data

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_2_bug/root/svj_Lambdad5_rinv2.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

In [3]:
# 2-1. Check the number of event for each branch
if GP.length == Jet.length == Event.length:
    print("{} events in the .root file.".format(GP.length))
else:
    print("\033[7;31m****** Please check your .root file. ******\033[0m")

19360 events in the .root file.


## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels

In [4]:
df_xdxdx_23 = myAnal_v3_old.analyze_xdxdx(GP, status=23)
df_xdxdx_71 = myAnal_v3_old.analyze_xdxdx(GP, status=71)

For status = 23, all events only include 2 particles.


IndexError: single positional indexer is out-of-bounds

##### Conclusion: The bug comes from `status = 71`.

#### A. Debug for `analyze_xdxdx(GP, status=71)`

In [5]:
# event 0 with only +-4900101
# looks like normal
dfGP = GP.dataframelize(0)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101)]
dfGP_DQ_Status

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
55,4900101,23,44,-1,57,58,10.0,507.201447,0.431692,-0.346168
56,-4900101,23,44,-1,59,59,10.0,496.481354,-1.542102,2.787818
57,4900101,51,55,-1,60,61,10.0,31.892319,-0.455998,-0.709771
59,-4900101,52,56,-1,63,64,10.0,492.740875,-1.542103,2.787818
60,4900101,51,57,-1,93,93,10.0,36.739040,-0.223569,-0.881083
63,-4900101,51,59,-1,71,71,10.0,454.629242,-1.542552,2.809824
71,-4900101,52,63,-1,77,77,10.0,348.165131,-1.542562,2.808619
77,-4900101,52,71,-1,87,88,10.0,229.068954,-1.545240,2.807806
87,-4900101,51,77,-1,106,106,10.0,290.054260,-1.557732,2.802647
93,4900101,71,60,-1,107,116,10.0,36.739040,-0.223569,-0.881083


In [6]:
dfGP_DQ_Status.shape

(11, 10)

In [7]:
dfGP.loc[107:116]

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
107,-4900213,1,93,106,-1,-1,20.0,38.282505,-0.094499,-0.729459
108,4900211,1,93,106,-1,-1,20.0,86.543503,0.348117,-0.270993
109,4900113,83,93,106,208,209,20.0,184.155991,0.487106,-0.323426
110,4900113,83,93,106,210,211,20.0,110.631760,0.452069,-0.277508
111,4900111,83,93,106,215,216,20.0,50.065376,0.439273,-0.320355
112,4900113,83,93,106,226,227,20.0,30.819977,0.360323,-0.446562
113,4900113,84,93,106,237,238,20.0,6.623464,-2.010812,1.722109
114,4900113,84,93,106,239,240,20.0,38.489616,-1.481691,2.820825
115,-4900213,1,93,106,-1,-1,20.0,125.432510,-1.489871,2.803738
116,4900211,1,93,106,-1,-1,20.0,319.203583,-1.551089,2.802279


In [8]:
dfGP.loc[[208, 209, 210, 211, 215, 216, 226, 227, 237, 238, 239, 240]]

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
208,-5,91,109,-1,337,337,4.80,146.694351,0.466185,-0.283111
209,5,91,109,-1,336,336,4.80,38.043068,0.559259,-0.479471
210,-1,91,110,-1,212,213,0.33,38.078274,0.492769,-0.031597
211,1,91,110,-1,214,214,0.33,74.279724,0.420975,-0.402630
215,-3,91,111,-1,219,219,0.50,47.494270,0.298363,-0.287556
216,3,91,111,-1,217,218,0.50,3.027939,1.736046,-0.860636
226,-2,91,112,-1,228,229,0.33,13.050719,0.674982,-1.125711
227,2,91,112,-1,230,230,0.33,22.231667,0.083286,-0.068916
237,-5,91,113,-1,322,322,4.80,3.592106,-0.102927,-0.577443
238,5,91,113,-1,321,321,4.80,9.405421,-1.663272,2.011025


In [9]:
dfGP.loc[[337, 336, 212, 213, 214, 219]]

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
337,-5,71,208,-1,338,340,4.80,146.694351,0.466185,-0.283111
336,5,71,209,-1,338,340,4.80,38.043068,0.559259,-0.479471
212,-1,51,210,-1,357,357,0.33,37.775600,0.474680,-0.047849
213,21,51,210,-1,356,356,0.00,1.713186,0.822432,0.007788
214,1,52,211,-1,355,355,0.33,72.772942,0.420976,-0.402628
219,-3,52,215,-1,222,222,0.50,45.284691,0.298372,-0.287558


##### Conclusion: Event 0 is fine.

#### B. Try to find the problem

In [10]:
test = []

for i in range(GP.length):
    dfGP = GP.dataframelize(i)
    dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 71)]
    if dfGP_DQ_Status.shape[0] == 0:
        test.append(i)
test

[7777]

In [11]:
# event 7777 with +-4900101 and Status = 71
# there are no +-4900101 with status = 71 at event 7777
dfGP = GP.dataframelize(7777)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 71)]
print(f'shape = {dfGP_DQ_Status.shape}', f'and {dfGP_DQ_Status.shape[0]} event')
dfGP_DQ_Status

shape = (0, 10) and 0 event


,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi


In [12]:
m1 = dfGP_DQ_Status.iloc[0, 6]
m1

IndexError: single positional indexer is out-of-bounds

##### Conclusion: The bug is there is no `xd` and `xdx` with `Status = 71` at event 7777.

#### C. Try to debug

In [13]:
# event 7777 with only +-4900101
# the dark quark pair +-4900101 is stable final state
dfGP = GP.dataframelize(7777)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101)]
dfGP_DQ_Status

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
612,4900101,23,482,-1,614,615,10.0,375.463379,2.102583,-1.898968
613,-4900101,23,482,-1,616,616,10.0,602.383545,0.064803,1.005123
614,4900101,51,612,-1,637,637,10.0,223.863174,1.095054,-1.403303
616,-4900101,52,613,-1,619,619,10.0,559.154785,0.064822,1.005122
619,-4900101,52,616,-1,622,622,10.0,540.150024,0.064833,1.005123
622,-4900101,52,619,-1,623,624,10.0,525.238647,0.064841,1.005121
623,-4900101,51,622,-1,634,634,10.0,387.097321,-0.013618,1.050996
634,-4900101,1,623,-1,-1,-1,10.0,255.655472,-0.011533,1.049971
637,4900101,52,614,-1,643,643,10.0,221.279816,1.095092,-1.403312
643,4900101,52,637,-1,661,661,10.0,214.782104,1.095196,-1.403338


In [14]:
# event 7777 with +-4900101 and +-4900021
# the dark quark pair +-4900101 and gv (+-4900021) are stable final state
dfGP = GP.dataframelize(7777)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) | (abs(dfGP['PID']) == 4900021)]
dfGP_DQ_Status = dfGP_DQ_Status[dfGP_DQ_Status['Status'] == 1]
dfGP_DQ_Status

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
634,-4900101,1,623,-1,-1,-1,10.0,255.655472,-0.011533,1.049971
655,4900021,1,645,-1,-1,-1,0.0,14.126121,0.085805,0.712336
665,4900021,1,647,-1,-1,-1,0.0,1.246145,2.824259,0.808135
666,4900021,1,647,-1,-1,-1,0.0,7.237934,-0.454872,-3.128381
668,4900021,1,663,-1,-1,-1,0.0,53.842560,2.401923,3.124650
672,4900021,1,639,-1,-1,-1,0.0,19.561600,2.643754,-2.907156
673,4900021,1,662,-1,-1,-1,0.0,10.043142,1.989169,-2.639540
674,4900021,1,658,-1,-1,-1,0.0,4.864041,1.926429,-2.742093
675,4900021,1,658,-1,-1,-1,0.0,5.530111,3.164004,0.655175
676,4900021,1,667,-1,-1,-1,0.0,3.857526,1.042714,0.543829


##### Conclusion: +-4900101 and many 4900021 are stable final state.
Therefore, I need to filter out and check them.

In [15]:
# calculate the invariant mass of +-4900101 with status = 1
# event 7777 with +-4900101 and Status = 71
dfGP = GP.dataframelize(7777)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 1)]
dfGP_DQ_Status
m1, pt1 = dfGP_DQ_Status.iloc[0, 6], dfGP_DQ_Status.iloc[0, 7]
eta1, phi1 = dfGP_DQ_Status.iloc[0, 8], dfGP_DQ_Status.iloc[0, 9]
m2, pt2 = dfGP_DQ_Status.iloc[1, 6], dfGP_DQ_Status.iloc[1, 7]
eta2, phi2 = dfGP_DQ_Status.iloc[1, 8], dfGP_DQ_Status.iloc[1, 9]

myAnal_v3.M(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2)

462.64557421338634

##### Conclusion: The reconstruction is bad, since there are many `gv` radiation.

In [16]:
dfGP[dfGP['Status'] == 1].iloc[20:40]

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
690,4900021,1,679,-1,-1,-1,0.00000,5.223913,2.051219,-0.701502
691,4900021,1,657,-1,-1,-1,0.00000,6.692794,1.906346,-1.552232
692,4900021,1,677,-1,-1,-1,0.00000,46.552296,1.067052,-0.769513
693,4900021,1,677,-1,-1,-1,0.00000,5.949739,1.750201,-0.369231
694,4900021,1,682,-1,-1,-1,0.00000,3.024438,1.362232,2.427163
695,4900021,1,653,-1,-1,-1,0.00000,17.456381,0.715772,1.240327
696,4900021,1,653,-1,-1,-1,0.00000,2.206179,0.159763,-0.984920
697,4900021,1,654,-1,-1,-1,0.00000,15.810166,-0.216263,1.170011
698,4900021,1,670,-1,-1,-1,0.00000,1.017656,0.062337,2.759575
699,4900021,1,670,-1,-1,-1,0.00000,16.615667,2.064426,-0.819149


### D. Is there other dark sector?
PID = 4900102-4900108 and 4900022

In [17]:
test = 0

for i in range(GP.length):
    dfGP = GP.dataframelize(i)
    pid = dfGP['PID'].to_numpy()
    pid_2 = pid[(pid == 4900102) | (pid == 4900022)]
    test += pid_2.shape[0]
    if pid_2.shape[0] != 0:
        print(i, pid_2)
        
if test == 0:
    print('There is no 4900102 or 4900022.')
else:
    print('Existence of 4900102 or 4900022.')

There is no 4900102 or 4900022.


#### E. Debug!!!

In [18]:
# event 7777 with +-4900101 and Status = 71
# there are no +-4900101 with status = 71 at event 7777
for i in range(7775, 7779):
    dfGP = GP.dataframelize(i)
    dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 71)]
    if dfGP_DQ_Status.shape[0] != 2:
        print(i)
        dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 1)]
    m1 = dfGP_DQ_Status.iloc[0, 6]
    print(f'event {i}: {m1}')
dfGP_DQ_Status

event 7775: 10.0
event 7776: 10.0
7777
event 7777: 10.0
event 7778: 10.0


,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
389,4900101,71,359,-1,403,414,10.0,80.751884,-2.217280,2.743159
402,-4900101,71,388,-1,403,414,10.0,396.666016,0.064742,-0.221863


In [19]:
df_xdxdx_23 = myAnal_v3.analyze_xdxdx(GP, status=23)
df_xdxdx_71 = myAnal_v3.analyze_xdxdx(GP, status=71)

For status = 23, all events only include dark quark pair with status 23.
* Skip event 7777
The length of 1 events is not equal to 2 particles.
[7777]


##### Conclusion: Successfully debug for `Lambdad = 5` and `rinv = 0.2` data!!

## 2. Import .root File and Load the Data via class
Use `Lambdad = 5` and `rinv = 1` data

In [20]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_2_bug/root/svj_Lambdad5_rinv10.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

In [21]:
# 2-1. Check the number of event for each branch
if GP.length == Jet.length == Event.length:
    print("{} events in the .root file.".format(GP.length))
else:
    print("\033[7;31m****** Please check your .root file. ******\033[0m")

19286 events in the .root file.


## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels

In [22]:
df_xdxdx_23 = myAnal_v3_old.analyze_xdxdx(GP, status=23)
df_xdxdx_71 = myAnal_v3_old.analyze_xdxdx(GP, status=71)

For status = 23, all events only include 2 particles.


IndexError: single positional indexer is out-of-bounds

#### A. Debug for `analyze_xdxdx(GP, status=71)`

In [23]:
test = []

for i in range(GP.length):
    dfGP = GP.dataframelize(i)
    dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 71)]
    if dfGP_DQ_Status.shape[0] == 0:
        test.append(i)
test

[623]

In [24]:
# event 623 with +-4900101 and Status = 71
# there are no +-4900101 with status = 71 at event 623
dfGP = GP.dataframelize(623)
dfGP_DQ_Status = dfGP[(abs(dfGP['PID']) == 4900101) & (dfGP['Status'] == 71)]
print(f'shape = {dfGP_DQ_Status.shape}', f'and {dfGP_DQ_Status.shape[0]} event')
dfGP_DQ_Status

shape = (0, 10) and 0 event


,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi


##### Conclusion: The bug is there is no `xd` and `xdx` with `Status = 71` at event 623.

#### B. Debug!!!

In [25]:
df_xdxdx_23 = myAnal_v3.analyze_xdxdx(GP, status=23)
df_xdxdx_71 = myAnal_v3.analyze_xdxdx(GP, status=71)

For status = 23, all events only include dark quark pair with status 23.
* Skip event 623
The length of 1 events is not equal to 2 particles.
[623]


##### Conclusion: Successfully debug for `Lambdad = 5` and `rinv = 0.2` data!!